In [ ]:
from math import *
import numpy as np
import matplotlib.pyplot as plt 
from keras.models import Model
from keras.utils import plot_model      #Useful to plot the scheme of the NN
from sklearn.model_selection import train_test_split as test_split
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import cv2                              #Library for computer vision problems

## Machine learning assignment 3

We now generate ourself some images with a circle or a rectangle, of random color, in a random position.

Three different modes of generating the images are implemented:

*   A single shape per figure
*   Multiple shapes mixed in each figure
*   A single figure keeping track of the "bounding box"



### Additional Exercise
1. Try adding some random noise in the image background
2. Try adding more classes such e.g. Lines or Ellipses
3. If we have more categories (let say N) we should use a categorical label that is a vector of length N with 1 on the category(/ies) the image belong to and 0 in the others. Try to build a categorical label for two categories 
4. Expand the categorical label to  Ellispes or  Lines  and possibly also non exclusive categories such has 2D vs 1D objects
5. Try adding/removing convolutional layers, change the kernel size, try to add dropout
6. Try changing the model to categorical labels, change loss function from binary_crossentropy to categorical_crossentropy, and use softmax activation instead of sigmoid


In [ ]:
noise = 2        #add a noise level to the background
simple = True    #single shape per figure
mixed  = False   #multi shapes per figure
withBB = False   #one shape with bounding boxes

def background():
  if noise == 1 :
    return np.array(np.random.rand(64,64,3)*20+100,np.uint8)
  elif noise == 2:
    return np.array(np.random.rand(64,64,3)*128+128,np.uint8)
  else :
    return np.zeros((64,64,3), np.uint8)

def randomColor():
  return (int(np.random.rand()*128+128),int(np.random.rand()*128+128),int(np.random.rand()*128+128))

def drawCircle(c,x,y,r):
  img = background()
  cv2.circle(img,(x,y),r,c, -1)
  return img,x-r,y-r,x+r,y+r   #return image and bounding box

def genCircle():
  return drawCircle(randomColor(),int(np.random.rand()*50)+10,int(np.random.rand()*50)+10,
                    int(np.random.rand()*6)+3)

def drawRectangle(c,x,y,w,h):
  img = background()
  cv2.rectangle(img,(x,y),((x+w),(y+h)), c, -1)
  return img,x,y,x+w,y+h #return image and bounding box

def genRectangle():
  return drawRectangle(randomColor(),int(np.random.rand()*40)+10,int(np.random.rand()*40)+10,
                       int(np.random.rand()*12)+5,int(np.random.rand()*12)+5)
  
def drawEllipse(c,x,y,axes,ang):
  img = background()
  cv2.ellipse(img,(x,y), axes, ang, 0, 360, c, -1)
  a = axes[0] // 2
  return img,x-a,y-a,x+a,y+a

def genEllipse():
  a = int(np.random.rand()*6)+3
  b = int(np.random.rand()*6)+3
  if a > b :
    axes = (a, b)
  else:
    axes = (b, a)
  return drawEllipse(randomColor(),int(np.random.rand()*40)+10,int(np.random.rand()*40)+10,
                       axes, int(np.random.rand()*360))
  

def genN(f,i): # generate multiple shapes
  img = background()
  for x in range(i):
    img+=f()[0] #discard bb info, take only image
  return img

def random_label(x):
  if x < 0.333:
    return [1, 0, 0]
  elif x > 0.666:
    return [0, 0, 1]
  else:
    return [0, 1, 0]

def random_fig(x):
  img = background()
  for i in range(x[0]):


nsamples = 5000

#produce figures with either a rectangle or a circle
if simple :
  targets = np.array(random_label(np.random.rand()) for x in range(nsamples))
  images = np.array([np.zeros((64,64,3), np.uint8) for x in range(nsamples)])


#produce figure with n rectangles and m circles
if mixed:
  targets=np.array([(int(np.random.rand()*4),int(np.random.rand()*4)) for x in range(nsamples) ])
  images=np.array([genN(genRectangle,targets[x,0])+genN(genCircle,targets[x,1]) for x in range(nsamples)])

if withBB : 
#produce figures with either a rectangle or a circle
  targets=np.array([np.random.rand()>0.5 for x in range(nsamples) ])
  imagesWithBB=[genCircle() if targets[x] else genRectangle() for x in range(nsamples)]
  images=np.array([imagesWithBB[x][0] for x in range(nsamples)])
  boundingBoxes=np.array([imagesWithBB[x][1:] for x in range(nsamples)])
